In [127]:
import keras
import cv2
import numpy as np
from glob import glob

# 相対パスを書くために、現在のディレクトリの位置を書き出す
import os
os.getcwd()


'd:\\My_programing\\python\\AI\\Training\\Net\\DataAugumentation'

In [128]:
def data_load(path, img_width, img_height, CLS):
    """ファイルパスからデータセットをロードするための関数"""
    xs = [] # 訓練データ用の空リスト
    ts = [] # データラベル用の空リスト
    paths = [] # ロードしたファイルパス用の空リスト
    img_read_err = [] # ロードできなかったファイルパス用の空リスト

    # データをロードするためにディレクトリを下りていく
    for dir_path_1 in glob(DirPath):
        for dir_path_2 in glob(dir_path_1 + '/*'):
            for path in glob(dir_path_2 + '/*'):
                #print(path, 'を読み込みました。')

                # 訓練用画像を読み込む
                x = cv2.imread(path)
                if x is None: # もし，画像がロードできなかった場合
                    print(path, 'を読み込めませんでした．')
                    img_read_err.append(path)
                    continue
                else: # 画像がロードできた場合
                    x = cv2.resize(x, (img_width, img_height)).astype(np.float32)
                    x /= 255.
                    xs.append(x)
                
                # one-hot-labelを作成する
                t = np.zeros(num_classes)
                p = dir_path_1[dir_path_1.find('\\'):].strip('\\')
                for i, cls in enumerate(CLS):
                    if cls == p:
                        t[i] = 1

                ts.append(t)
                paths.append(path)

    xs = np.array(xs, dtype=np.float32)
    ts = np.array(ts, dtype=np.int)

    return xs, ts, paths, img_read_err

In [129]:
# LeNetを作成する
## Network
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, MaxPool2D, Input, BatchNormalization

def LeNet(img_width, img_height, num_classes):
    inputs = Input((img_height, img_width, 3))
    x = Conv2D(6, (5, 5), padding='valid', activation=None, name='conv1')(inputs)
    x = MaxPool2D((2, 2), padding='same')(x)
    x = Activation('sigmoid')(x)
    x = Conv2D(16, (5, 5), padding='valid', activation=None, name='conv2')(x)
    x = MaxPool2D((2, 2), padding='same')(x)
    x = Activation('sigmoid')(x)

    x = Flatten()(x)
    x = Dense(120, name='dense1', activation=None)(x)
    x = Dense(64, name='dense2', activation=None)(x)
    x = Dense(num_classes, activation='softmax')(x)

    model = Model(inputs=inputs, outputs=x, name='model')
    return model

In [130]:
def train(DirPath, img_size, cls_label):
    model = LeNet(img_size[0], img_size[1], len(cls_label)+1)

    for layer in model.layers:
        layer.trainable = True

    model.compile(
        loss = 'categorical_crossentropy',
        optimizer = keras.optimizers.SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True),
        metrics=['accuracy'])
    
    xs, ts, paths, _ = data_load(DirPath, img_size[0], img_size[1], cls_label)

    # training
    mb = 8
    mbi = 0
    train_ind = np.arange(len(xs))
    np.random.seed(0)
    np.random.shuffle(train_ind)

    for i in range(500):
        if mbi + mb > len(xs):
            mb_ind = train_ind[mbi:]
            np.random.shuffule(train_ind)
            mb_ind = np.hstack((mb_ind, train_ind[:(mb-(len(xs)-mbi))]))
            mbi = mb - (len(xs) - mbi)
        else:
            mb_ind = train_ind[mbi: mbi+mb]
            mbi += mb

        x = xs[mb_ind]
        t = ts[mb_ind]

        loss, acc = model.train_on_batch(x=x, y=t)
        print('iter >>', i+1, ',loss >>', loss, 'accuracy >>', acc)

    model.save('LeNet.h5')
    return 0

In [131]:
# データを読み込むフォルダを指定する
DirPath = '../../../DataSet/AngleDetection/training/*'
#print(DirPath)

num_classes = 36
img_width, img_height = 64, 64
CLS = np.arange(0, 175, 5).astype('str')

#train(DirPath, img_size=(64, 64), cls_label=CLS)
train(DirPath, img_size=(img_width, img_height), cls_label=CLS)

y >> 0.125
iter >> 77 ,loss >> 3.120249 accuracy >> 0.0
iter >> 78 ,loss >> 3.1556344 accuracy >> 0.0
iter >> 79 ,loss >> 3.5754528 accuracy >> 0.125
iter >> 80 ,loss >> 3.62537 accuracy >> 0.0
iter >> 81 ,loss >> 3.6660626 accuracy >> 0.0
iter >> 82 ,loss >> 3.5605164 accuracy >> 0.0
iter >> 83 ,loss >> 3.6067295 accuracy >> 0.0
iter >> 84 ,loss >> 3.1216013 accuracy >> 0.0
iter >> 85 ,loss >> 3.618331 accuracy >> 0.0
iter >> 86 ,loss >> 3.5423725 accuracy >> 0.0
iter >> 87 ,loss >> 3.557204 accuracy >> 0.0
iter >> 88 ,loss >> 3.55056 accuracy >> 0.0
iter >> 89 ,loss >> 3.055048 accuracy >> 0.0
iter >> 90 ,loss >> 3.6598592 accuracy >> 0.0
iter >> 91 ,loss >> 3.528348 accuracy >> 0.125
iter >> 92 ,loss >> 3.6100626 accuracy >> 0.0
iter >> 93 ,loss >> 3.6029131 accuracy >> 0.0
iter >> 94 ,loss >> 3.5489721 accuracy >> 0.125
iter >> 95 ,loss >> 3.604447 accuracy >> 0.0
iter >> 96 ,loss >> 3.1702156 accuracy >> 0.0
iter >> 97 ,loss >> 3.552099 accuracy >> 0.0
iter >> 98 ,loss >> 3.503419

0